In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
sys.path.append("python_lib")

In [3]:
import pandas as pd
import nnsplit
from nnsplit import train, utils, models

In [4]:
paragraphs = train.xml_to_paragraphs("train_data/dewiki-20180920-corpus.xml", max_n_paragraphs=100_000)

In [5]:
pd.Series([len(x) for x in paragraphs]).mean()

804.17998

In [6]:
tokenizer = nnsplit.NNSplit("de")

In [7]:
%%time
_ = tokenizer.split(paragraphs, batch_size=2048)

CPU times: user 1min 22s, sys: 2.5 s, total: 1min 25s
Wall time: 51.2 s
